# Visualize batches, apply pyCombat and visualize again

- This notebook is to visualize batch effects of High-Content screening assays, try to correct those effects using pyCombat [reference here], and visualize it again with UMAP to check if the effect was corrected.


github@fefossa

In [1]:
import pandas as pd
import easygui as eg
import sys

sys.path.append(r"C:\Users\Fer\Documents\GitHub")

from scripts_notebooks_fossa.pycombat_umap import combat_util

%load_ext autoreload
%autoreload 2

c:\Users\Fer\miniconda3\envs\pycytominer\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Fer\miniconda3\envs\pycytominer\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\Fer\miniconda3\envs\pycytominer\lib\site-packages\umap\distances.py:108

# 0. Inputs

In [2]:
myfile = eg.fileopenbox(msg="Choose a file", default=r"F:")
print('Filename', myfile)
df = pd.read_csv(myfile)
df.head()

Filename C:\Users\Fer\Desktop\example_batch effet\profiles\Test_batch_effect_visualization_normalized_negcon_feature_select.csv


,Unnamed: 0,Metadata_Plate,Metadata_Well,Metadata_cell_count,Metadata_plate_map_name,Metadata_Concentration,Metadata_Cell,Metadata_Compound,Metadata_moa,Metadata_DILI-concern,...,Nuclei_Texture_Correlation_CorrPI_20_03_256,Nuclei_Texture_Correlation_CorrPI_5_00_256,Nuclei_Texture_Correlation_CorrPI_5_01_256,Nuclei_Texture_Correlation_CorrPI_5_02_256,Nuclei_Texture_Correlation_CorrPI_5_03_256,Nuclei_Texture_InfoMeas1_CorrGFP_20_03_256,Nuclei_Texture_InfoMeas1_CorrGFP_5_00_256,Nuclei_Texture_InfoMeas1_CorrPI_20_03_256,Nuclei_Texture_InfoMeas2_CorrGFP_5_00_256,Nuclei_Texture_InfoMeas2_CorrPI_20_01_256
0,0,220607_092050_Plate_1,B10,379,platemap_dili,10.0,Huh7,Etoposide,topoisomerase II inhibition,Less-DILI-Concern,...,5.395341,2.176806,1.990091,48.387255,1.974474,1.212773,-2.488307,0.539402,2.946562,0.051344
1,1,220607_092050_Plate_1,B11,697,platemap_dili,0.0,Huh7,Vehicle-ETP,ETP-Control,None,...,0.836678,-0.784427,0.030035,-62.191553,-3.388379,-2.004516,-0.437987,-1.783197,0.941576,2.066243
2,2,220607_092050_Plate_1,B3,544,platemap_dili,1.0,Huh7,Aspirin,COX inhibitor,Less-DILI-Concern,...,-2.208579,9.829729,6.816155,196.896797,23.537049,-13.271161,-0.979537,-3.876753,3.364471,3.272119
3,3,220607_092050_Plate_1,B4,584,platemap_dili,10.0,Huh7,Aspirin,COX inhibitor,Less-DILI-Concern,...,-0.096581,8.619137,5.273320,160.213646,18.685630,-14.791844,-1.857842,-4.521746,4.195629,3.709254
4,4,220607_092050_Plate_1,B5,561,platemap_dili,0.2,Huh7,Amiodarone,sodium/potassium-ATPase inhibitor,Most-DILI-Concern,...,-0.421291,6.718631,4.534543,113.441957,13.603080,-10.846040,-0.823910,-3.032397,3.062944,2.885333


In [3]:
output_path = eg.diropenbox(msg="Choose an output folder", default=r"X:")
print('Path to save the profile', output_path)

Path to save the profile C:\Users\Fer\Desktop\example_batch effet\profiles


## 0. UMAP parameters and hover cols

### UMAP

In [4]:
n_neighbors_input = 15
min_dist_input = 0.5
metric = 'cosine'

### Define hover columns to always have it on the plots

In [9]:
hover_list = ['Metadata_Plate', 'Metadata_Well', 'Metadata_Compound', 'Metadata_Concentration']

### Define a new column for the labels

In [10]:
cols_to_join = ["Metadata_Compound", "Metadata_Concentration"]

df, new_col = combat_util.col_generator(df, cols_to_join = cols_to_join)

Concentration
Names of the compounds + concentration:  ['Etoposide 10.0' 'Vehicle-ETP 0.0' 'Aspirin 1.0' 'Aspirin 10.0'
 'Amiodarone 0.2' 'Amiodarone 2.0' 'Cyclophosphamide 1.0'
 'Cyclophosphamide 10.0' 'Etoposide 1.0' 'Non-treated 0.0' 'Lactose 1.0'
 'Lactose 10.0' 'Lovastatin 1.0' 'Lovastatin 10.0' 'Orphenadrine 1.0'
 'Orphenadrine 10.0' 'Tetracycline 1.0' 'Tetracycline 10.0' 'DMSO 1.4']


# 1. Plot to check for batch effects

- We plot columns that usually has batch effects like Plate (if they were read on different days), Well position

## Plate

In [12]:
df_plot = combat_util.generate_x_y_umap(df, n_neighbors=n_neighbors_input, min_dist=min_dist_input, metric=metric)
combat_util.plot_umap(df_plot, 
                      color_col='Metadata_Plate',
                      hover_cols=hover_list)
combat_util.plot_umap(df_plot, color_col='Metadata_Compound', 
                      hover_cols=hover_list,
                      size=True, size_col = "Metadata_Concentration",
                      dili_color=True
                      )

# 2. Apply pyCombat

- Based on the batch effects you could visually identify above, choose the batch_col you'd like to correct 

In [13]:
batch_col = "Metadata_Plate"

In [14]:
df_plates = combat_util.pycombat_generator(df, batch_column = batch_col, well_column = 'Metadata_Well')

Found 3 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data


In [15]:
output_name = "Test_batch_effect_visualization_normalized_negcon_feature_select_pycombat"

df_plates.to_csv(rf"{output_path}\{output_name}.csv" ,index=False)

# 3. Plot after batch correction

In [17]:
df_plot = combat_util.generate_x_y_umap(df_plates, n_neighbors=n_neighbors_input, min_dist=min_dist_input, metric=metric)
combat_util.plot_umap(df_plot, 
                      color_col='Metadata_Plate',
                      hover_cols=hover_list)
combat_util.plot_umap(df_plot, color_col='Metadata_Compound', split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col = "Metadata_Concentration",
                      dili_color=True
                      )

## 3D

In [18]:
df_3d = combat_util.generate_x_y_umap(df_plates, n_neighbors=n_neighbors_input, min_dist=min_dist_input, metric=metric, n_components=3)

In [19]:
combat_util.plot_umap_3d(df_3d, color_col="Metadata_Compound", split_df = False, split_column = None, np = None,
                      hover_cols=hover_list,
                      size=True, size_col = "Metadata_Concentration",
                      dili_color=True,
                      discrete=True
                      )